# Description

READ ME

# Boilerplate

## Dependencies

In [0]:
%pip install databricks-sdk==0.41.0
%restart_python

In [0]:
import mlflow
from databricks.feature_engineering import FeatureEngineeringClient

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import OnlineTable, OnlineTableSpec, OnlineTableSpecTriggeredSchedulingPolicy

## Parameters

In [0]:
dbutils.widgets.text("catalog_name", "", "00 - Catalog Name")
dbutils.widgets.text("schema_name", "", "01 - Schema Name")

In [0]:
params = dbutils.widgets.getAll()

for key, value in params.items():
  assert value != "", f"Parameter {key} is empty"

locals().update(params)

# Main

## Publish Online Tables

In [0]:
fe = FeatureEngineeringClient()
ws = WorkspaceClient()

# Create an online table
customer_weekly_spec = OnlineTableSpec(
  primary_key_columns = ["customer_id", "event_week"],
  source_table_full_name = f"{catalog_name}.{schema_name}.customer_weekly_features",
  run_triggered=OnlineTableSpecTriggeredSchedulingPolicy.from_dict({'triggered': 'true'}),
  perform_full_copy=True)

customer_weekly_spec_online_table = OnlineTable(
  name=f"{catalog_name}.{schema_name}.customer_weekly_features_online",
  spec=customer_weekly_spec)

try:
  ws.online_tables.create_and_wait(table=customer_weekly_spec_online_table)
except Exception as e:
    print(e)
  
# Create an online table
customer_demographics_spec = OnlineTableSpec(
  primary_key_columns = ["customer_id"],
  source_table_full_name = f"{catalog_name}.{schema_name}.customer_demographics",
  run_triggered=OnlineTableSpecTriggeredSchedulingPolicy.from_dict({'triggered': 'true'}),
  perform_full_copy=True)

customer_demographics_online_table = OnlineTable(
  name=f"{catalog_name}.{schema_name}.customer_demographics_online",
  spec=customer_demographics_spec)

try:
  ws.online_tables.create_and_wait(table=customer_demographics_online_table)
except Exception as e:
    print(e)

## Serve Model

In [0]:
mlflow.set_registry_uri("databricks-uc")
client = mlflow.tracking.MlflowClient()

registered_model_name = f"{catalog_name}.{schema_name}.invoice_increase_model"
model = client.get_model_version_by_alias(name=registered_model_name, alias="champion")
model_version = model.version
model_version

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

# Create endpoint
endpoint_name = f"{catalog_name}_{schema_name}_invoice_increase"

try:
  status = ws.serving_endpoints.create_and_wait(
    name=endpoint_name,
    config = EndpointCoreConfigInput(
      served_entities=[
        ServedEntityInput(
            entity_name=f"{catalog_name}.{schema_name}.invoice_increase_model",
            entity_version=model_version,
            scale_to_zero_enabled=True,
            workload_size="Small"
        )
      ]
    )
  )
  print(status)
except Exception as e:
  if "already exists" in str(e):
    print(f"Not creating endpoint {endpoint_name} since it already exists.")
  else:
    raise e

## Score Model Online

In [0]:
test_df = spark.sql(f"""
    SELECT 
      customer_id
      , billing_month
    FROM {catalog_name}.{schema_name}.invoices""").toPandas()

test_df

In [0]:
test_df[:10].to_dict(orient="records")

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")
response = client.predict(
    endpoint=endpoint_name,
    inputs={
        #"dataframe_records": test_df[:10].to_dict(orient="records")
        "dataframe_records": [{'customer_id': '1400', 'billing_month': "2024-07-01"},
                              {'customer_id': '1609', 'billing_month': "2025-05-01"}]
    },
)

response